

# Libraries and Setup



In [1]:
%pip install -q -r C:\Users\Boomin\Documents\GitHub\Arabic-Dialect-Detector\requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification, Trainer, TrainingArguments, BitsAndBytesConfig, DataCollatorWithPadding
from peft import LoraConfig, TaskType, get_peft_model
from datasets import load_dataset, DatasetDict
from huggingface_hub import login
from dotenv import load_dotenv
#from google.colab import userdata

import numpy as np
import pandas as pd
import torch
import os

C:\Users\Boomin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Users\Boomin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.trans

In [5]:
load_dotenv()
hf_token = os.getenv('HUGGINGFACE_TOKEN')
login(token=hf_token)

# Model Design

 ### Quantization configs

In [ ]:
# bnb library quantizes model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True
)

# peft config for quantized model
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, # SEQ_CLS for dialect detection, SEQ_2_SEQ_LM for translation
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.5
)

### Model Setup

In [ ]:
# define model
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-mix"
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           quantization_config=bnb_config,
                                                           num_labels=5,
                                                           device_map ="auto")

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

### Data

In [6]:
# define tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                        do_lower_case=False,
                                        use_fast=True)
special_tokens_dict = {
    'cls_token': '[CLS]',
    'sep_token': '[SEP]',
    'pad_token': '[PAD]',
    'mask_token': '[MASK]'
}
tokenizer.add_special_tokens(special_tokens_dict)

model.resize_token_embeddings(len(tokenizer))

NameError: name 'model_name' is not defined

In [ ]:
# load data
training_data_path = r"C:\Users\Boomin\Documents\GitHub\Arabic-Dialect-Detector\src\data\Arabic_dialect.csv"
dataset = load_dataset("csv", data_files=training_data_path)

dataset = dataset.rename_column("result", "label")
dataset = dataset.rename_column("t", "text")

dataset.set_format(type="torch", columns=["text", "label"])

In [ ]:
# split data
train_test = dataset["train"].train_test_split(test_size=0.4, seed= 42)
test_val = train_test["test"].train_test_split(test_size=0.5, seed= 42)

columns_to_remove = ['Date','User', 'Tweet']
# format data into DatsetDict
dataset = DatasetDict({
    "train": train_test["train"].remove_columns(columns_to_remove),
    "validation": test_val["train"].remove_columns(columns_to_remove),
    "test": test_val["test"].remove_columns(columns_to_remove)
})
dataset

In [ ]:
# Set the pad token if it's missing
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id
tokenizer.padding_side = "right"


# Tokenization function
max_l = 128
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128)


# Tokenize the dataset
tokenized_datasets = dataset.map(preprocess_function,
                                 batched=True,
                                 load_from_cache_file=False,
                                 batch_size = 16)

print(tokenized_datasets)
# tokenized_datasets['train'][0]

In [ ]:
columns_to_remove = ['label', 'text']

tokenized_datasets = tokenized_datasets.map(
    lambda examples: {"labels": examples["label"]},
)

tokenized_datasets = DatasetDict({
    "train": tokenized_datasets["train"].remove_columns(columns_to_remove),
    "validation": tokenized_datasets["validation"].remove_columns(columns_to_remove),
    "test": tokenized_datasets["test"].remove_columns(columns_to_remove),
})
tokenized_datasets

In [ ]:
print(f"Padding Token: {tokenizer.pad_token}")
print(tokenizer.special_tokens_map)
print(f"Padding Token ID: {tokenizer.pad_token_id}")

# Train

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)


In [ ]:

print(tokenized_datasets["train"][0])  # Check tokenized example


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
)

# Initialize the Trainer, adding the compute_metrics argument
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator
)

# help memory please????
torch.cuda.empty_cache()
model.gradient_checkpointing_enable()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


# Fine-tune the model
trainer.train()

Eval

In [ ]:
# Evaluate on the test set
results = trainer.evaluate(tokenized_datasets["test"])
print(results)
